Данный ноутбук является второй эпостасией попыток найти решение.

Дабы не корректировать первый, зафиксировать те результаты, которых я достиг и возвращаться к ним при необходимости.

Здесь для начала хочу получше разобраться с данными и фичами.

Затем все таки нормализовать уменьшить размерность данных, нормализовать их, обучить алгоритмы LogReg, SVM, Decision Tree, CatBoost и произвести стэкинг и попытьася улучшить скор.

### Импорты

In [90]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from category_encoders.cat_boost import CatBoostEncoder

import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score, classification_report, f1_score

from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

import time
import warnings
from tqdm import tqdm

from imblearn.over_sampling import SMOTE
warnings.filterwarnings("ignore")

### Загрузка данных

In [39]:
start = time.time()
train = pd.read_csv('./data/train.csv')
elapsed = time.time() - start
print(f'Time is: {elapsed} seconds')

Time is: 19.816019296646118 seconds


In [40]:
start = time.time()
test = pd.read_csv('./data/test.csv')
elapsed = time.time() - start
print(f'Time is: {elapsed} seconds')

Time is: 5.831754207611084 seconds


### Оптимизация

In [41]:
def optimize_data(df):
    # оптимизируем числовые данные
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')

    # Оптимизируем даты
    df["created_at"] = pd.to_datetime(df["created_at"])
    df["start_dt"] = pd.to_datetime(df["start_dt"])

    return df

In [42]:
# Размер трейна до оптимизации
print(f"Размер датасета до оптимизации: {train.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
train = optimize_data(train)
# Размер трейна после оптимизации
print(f"Размер датасета после оптимизации: {train.memory_usage(deep=True).sum() / (1024**2):.2f} MB")

Размер датасета до оптимизации: 3231.96 MB
Размер датасета после оптимизации: 2150.51 MB


In [43]:
# Размер теста до оптимизации
print(f"Размер датасета до оптимизации: {test.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
test = optimize_data(test)
# Размер теста после оптимизации
print(f"Размер датасета после оптимизации: {test.memory_usage(deep=True).sum() / (1024**2):.2f} MB")

Размер датасета до оптимизации: 971.89 MB
Размер датасета после оптимизации: 619.54 MB


### EDA

In [44]:
# will drop the closed_at, because it isn't in test
train.drop(['closed_at'], inplace=True, axis=1)

In [45]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4036207 entries, 0 to 4036206
Data columns (total 44 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   monthly_income         float32       
 1   work_experience        float32       
 2   requested_sum          float32       
 3   main_agreement_amount  float32       
 4   main_agreement_term    float32       
 5   requested_period_days  float32       
 6   requested_amount       float32       
 7   req_app_amount         float32       
 8   approved_amount        float32       
 9   period_days            float32       
 10  days_finish_loan       float32       
 11  ag                     float32       
 12  cnt_ext                float32       
 13  term                   float32       
 14  price                  float32       
 15  elecs_sum              float32       
 16  recurents_sum          float32       
 17  tamount                float32       
 18  issues                

Я не разбирался с пропусками детально, а просто дропал столбики, где пропусков было прям много.

Такие значения можно не дропать, а заменить на 0 / 'unknown'

Взглянем на столбики, в которых пропущены значения

In [9]:
train.isnull().sum()

monthly_income               207
work_experience          2648360
requested_sum            3546390
main_agreement_amount          0
main_agreement_term            0
requested_period_days     441618
requested_amount          414836
req_app_amount            414836
approved_amount                0
period_days                    0
days_finish_loan               0
ag                             0
cnt_ext                  3582556
term                     3582556
price                    3582816
elecs_sum                   5794
recurents_sum               5794
tamount                     1774
issues                         0
principal                      0
interest                       0
overdue_interest               0
overdue_fee                    0
nbki_score                 84476
payment_frequency              0
status                         0
loan_id                        0
client_id                      0
source                         0
first_source                   0
interface 

Есть небольшие пропуски в некоторых столбцах, а в некоторых прям ПРИЛИЧНЫЕ.

Отличились следующие фичи:
- work_experience - стаж - 65% пропусков
- requested_sum - запрашиваемая сумма клиента для займа - 88% пропусков
- cnt_ext - кол-во пролонгаций по займу - 89% пропусков
- term - срок пролонгации (список) - 89% пропусков
- price -  цена пролонгации (список) - 89% пропусков
- start_dt  -  дата начала (список) пролонгаций по займу - 89% пропусков
- contact_cases - кол-во обращений клиента с коллекшн - 94% пропусков

Надо их разобрать.

In [46]:
some_null = ['monthly_income', 'elecs_sum', 'recurents_sum', 'tamount', 'nbki_score','requested_period_days', 'requested_amount', 'req_app_amount']
many_null = ['work_experience','requested_sum','cnt_ext','term','price','contact_cases','start_dt']

In [47]:
train[some_null]

,monthly_income,elecs_sum,recurents_sum,tamount,nbki_score,requested_period_days,requested_amount,req_app_amount
0,-0.000735,-0.063933,-0.143015,-0.434972,1.837623,-0.389028,-1.053530,-0.629296
1,-0.000742,-0.063933,-0.143015,-0.434972,2.262600,-1.294731,-0.436189,-0.020140
2,-0.000742,-0.063933,-0.143015,-0.429509,2.262600,-1.113590,-0.259806,0.153905
3,-0.000742,-0.063933,-0.143015,-0.413118,2.262600,-1.385301,1.239449,1.633284
4,-0.000742,-0.063933,-0.143015,-0.320236,2.262600,-1.657011,0.357534,0.763061
...,...,...,...,...,...,...,...,...
4036202,-0.000747,-0.063933,-0.143015,-0.252851,0.866246,NaN,NaN,NaN
4036203,-0.000747,-0.063933,-0.143015,-0.708154,-1.180584,0.426103,-1.141721,-1.064407
4036204,-0.000739,-0.063933,-0.143015,0.202452,-0.100794,NaN,NaN,NaN
4036205,-0.000730,-0.063933,-0.143015,0.111392,0.367548,0.426103,-0.083423,-0.194184


Данные, где небольшое кол-во пропусков заполню средними значениями по данными фичам.

In [48]:
train[many_null]

,work_experience,requested_sum,cnt_ext,term,price,contact_cases,start_dt
0,0.952894,NaN,NaN,NaN,NaN,NaN,NaT
1,0.952894,NaN,NaN,NaN,NaN,NaN,NaT
2,0.952894,NaN,NaN,NaN,NaN,NaN,NaT
3,0.952894,NaN,NaN,NaN,NaN,NaN,2024-01-12 17:29:21
4,0.952894,NaN,1.407752,-0.612324,-0.705103,NaN,NaT
...,...,...,...,...,...,...,...
4036202,NaN,-0.592014,NaN,NaN,NaN,NaN,NaT
4036203,0.084114,NaN,NaN,NaN,NaN,NaN,NaT
4036204,NaN,-0.592014,NaN,NaN,NaN,NaN,NaT
4036205,0.084114,NaN,NaN,NaN,NaN,NaN,NaT


In [49]:
train['contact_cases'].max()

64.0


`Пролонгация займа - это возможность продлить срок возврата взятого кредита без необходимости полного его погашения.`

То есть, увеличение срока кредита, по тем или иным обстоятельствам.

Соответственно, я думаю, что фичи:
- cnt_ext - кол-во пролонгаций по займу - 89% пропусков
- term - срок пролонгации (список) - 89% пропусков
- price -  цена пролонгации (список) - 89% пропусков
- start_dt  -  дата начала (список) пролонгаций по займу - 89% пропусков

По сути будут сильнокоррелировать.

Необходимо пропуски заполнить 0, оставив только cnt_ext

- пропущенные в 'contact_cases' так же заполняем нулями.

- пропуски в 'work_experience' так же заполню нулями

#### Обработка пропусков

In [50]:
droplist = ['term','price','start_dt','client_type.1']
zeroslist = ['cnt_ext','contact_cases', 'work_experience', 'requested_sum'] + some_null

In [51]:
train.drop(axis=1, labels=droplist, inplace=True)
test.drop(axis=1, labels=droplist, inplace=True)

In [52]:
# заполняем нулями все подряд
train[zeroslist] = train[zeroslist].fillna(0)
test[zeroslist] = test[zeroslist].fillna(0)

In [53]:
train.isnull().sum()

monthly_income           0
work_experience          0
requested_sum            0
main_agreement_amount    0
main_agreement_term      0
requested_period_days    0
requested_amount         0
req_app_amount           0
approved_amount          0
period_days              0
days_finish_loan         0
ag                       0
cnt_ext                  0
elecs_sum                0
recurents_sum            0
tamount                  0
issues                   0
principal                0
interest                 0
overdue_interest         0
overdue_fee              0
nbki_score               0
payment_frequency        0
status                   0
loan_id                  0
client_id                0
source                   0
first_source             0
interface                0
type                     0
repayment_type           0
client_type              0
settlement               0
region                   0
gender                   0
loan_order               0
have_extension           0
c

### Обработка категориальных фичей

#### Бинарные признаки

In [57]:
# удаляем регион
# для трейна
train.drop(columns=['region'], inplace=True)
# для теста
test.drop(columns=['region'], inplace=True)

In [60]:
# кодируем бинарные признаки для трейна
label_enc_cols = ['gender']
for col in label_enc_cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])

In [61]:
# кодируем бинарные признаки для теста
label_enc_cols = ['gender']
for col in label_enc_cols:
    le = LabelEncoder()
    test[col] = le.fit_transform(test[col])

#### CatBoost Encoding

Города сильно различаются, поэтому **One-Hot Encoding не подойдёт** (слишком много новых фич).

CatBoost Encoding (учитывает вероятности, полезно для ML-моделей).

In [63]:
cat_enc = CatBoostEncoder()
train['settlement'] = cat_enc.fit_transform(train['settlement'], train['churn'])

In [64]:
# Применяем обученный энкодер к test (без .fit!)
test['settlement'] = cat_enc.transform(test['settlement'])

#### дата
Даты нельзя просто так закодировать.

От них нужно или отказаться или же выделить полезные числовые признаки.

Получим новые признаки, а исходные столбцы удалим

In [55]:
# для трейна
train['created_at'] = pd.to_datetime(train['created_at'])

# День, месяц, год, день недели
train['created_day'] = train['created_at'].dt.day
train['created_month'] = train['created_at'].dt.month
train['created_weekday'] = train['created_at'].dt.weekday  # 0 = Понедельник, 6 = Воскресенье

# Удаление изначальных столбцов
train.drop(columns=['created_at'], inplace=True)

In [56]:
# для теста
test['created_at'] = pd.to_datetime(test['created_at'])

# День, месяц, год, день недели
test['created_day'] = test['created_at'].dt.day
test['created_month'] = test['created_at'].dt.month
test['created_weekday'] = test['created_at'].dt.weekday  # 0 = Понедельник, 6 = Воскресенье

# Удаление изначальных столбцов
test.drop(columns=['created_at'], inplace=True)

In [66]:
# взглянем на обновленные категориальные фичи!
cat_cols = ['settlement','gender', 'created_day','created_month','created_weekday']
train[cat_cols]

,settlement,gender,created_day,created_month,created_weekday
0,0.293872,1,23,12,4
1,0.146936,0,29,6,5
2,0.293872,1,10,4,0
3,0.293872,0,1,10,6
4,0.097957,0,15,7,0
...,...,...,...,...,...
4036202,0.256162,0,3,5,2
4036203,0.263662,0,13,9,2
4036204,0.287153,1,13,12,2
4036205,0.275080,1,27,8,1


### Баланс классов

In [75]:
train['churn'].value_counts()

churn
0    2850077
1    1186130
Name: count, dtype: int64

In [76]:
# Разделяем классы
churn_0 = train[train['churn'] == 0]
churn_1 = train[train['churn'] == 1]

In [77]:
# **Undersampling:** случайно удаляем 831973 записей класса 0
churn_0_sampled = churn_0.sample(n=len(churn_0) - 831973, random_state=42)

In [78]:
# Объединяем обе выборки перед SMOTE
train_balancing = pd.concat([churn_0_sampled, churn_1], axis=0)

In [79]:
# **Oversampling:** добавляем 831974 записей класса 1 с помощью SMOTE
X_resampled, y_resampled = SMOTE(sampling_strategy=1.0, random_state=42).fit_resample(
    train_balancing.drop(columns=['churn']), train_balancing['churn'])

In [80]:
# Превращаем в датафрейм
balanced_train = pd.DataFrame(X_resampled, columns=train_balancing.drop(columns=['churn']).columns)
balanced_train['churn'] = y_resampled

In [84]:
# Проверяем баланс
print(balanced_train['churn'].value_counts())

churn
0    2018104
1    2018104
Name: count, dtype: int64


In [85]:
# сохраню отбалансированные данные на будущее себе, чтобы была возможность продолжить с этого шага
balanced_train.to_csv("balanced_train.csv", index=False)

### Нормализация данных

In [94]:
train = balanced_train

In [95]:
scaler = StandardScaler()

In [89]:
train_std = scaler.fit_transform(train)
test_std = scaler.fit_transform(test)

### Уменьшение размерности

In [91]:
pca = PCA(n_components=2)
pca.fit(train)
x0, x1 = pca.components_
sns.set(font_scale=1.5)
y = train["Churn"].map({0: False, 1: True})
fig = sns.pairplot(data=pd.concat([pd.DataFrame(data=train @ np.stack([x0, x1]).T, columns=["PC1", "PC2"]), y], axis=1),
                   x_vars="PC1",
                   y_vars="PC2",
                   hue="Churn",
                   markers=('^', 's'),
                   palette=["blue", "red"],
                   plot_kws={'s': 100, 'alpha': 0.5},
                   height=6)
fig.set(title="Customer churn visialization (PCA)")
fig.axes[0][0].axhline(y=0, color='black', lw=3, alpha=0.1)
fig.axes[0][0].axvline(x=0, color='black', lw=3, alpha=0.1);

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [92]:
train['Churn']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [93]:
type(train)

numpy.ndarray

На сегодня пока все, позже поковыряю данный ноутбук =)